### Physical model + Echo + Proof-of-trend (v0.2.0)

**What**: Swap in physical simulator, keep LZ & RR, add Echo half-life; add windowed trends, light RQA, robustness, and stats.

**Sweep**: flux ∈ {0.0, 2e-4, 8e-4}, T_STEPS=9000, seeds {7,21,37} (+ extended optional).

**Decisions**:
- LZ slope vs flux > 0 (p<0.01)
- RR slopes < 0 for ≥2/3 ε (p<0.05)
- Echo half-life decreases with flux for ≥2/3 seeds and overall slope < 0 (p<0.05)

Outputs: `outputs/echo_half_life.csv`, `outputs/window_trends.csv`, `outputs/rqa_summary.csv`, `outputs/stats_summary.md`, plots in `outputs/`.


### 1) Imports & project paths

In [1]:

from __future__ import annotations
import os, time, zlib, warnings
from pathlib import Path
from typing import Dict, Iterable, Iterator, Tuple

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.filterwarnings("ignore")

def find_project_root(start: Path | None = None) -> Path:
    """Walk up until we find a .git directory; fall back to CWD."""
    p = Path.cwd() if start is None else Path(start)
    while True:
        if (p / ".git").exists():
            return p
        if p.parent == p:
            return Path.cwd()
        p = p.parent

PROJECT_ROOT = find_project_root()
os.chdir(PROJECT_ROOT)
OUTPUTS = PROJECT_ROOT / "outputs"
OUTPUTS.mkdir(parents=True, exist_ok=True)

print(f"Project root: {PROJECT_ROOT}")
print(f"Outputs dir:  {OUTPUTS}")



Project root: /Users/nadjamuller/number1/setup
Outputs dir:  /Users/nadjamuller/number1/setup/outputs


### 2) Parameters

In [2]:

T_STEPS     = 9000
FLUX_LEVELS = [0.0, 2e-4, 8e-4]
SEEDS       = [7, 21, 37]
EXTRA_SEEDS = [5, 11, 101]   # optional

RR_SCALES = (0.1, 0.3, 0.5)  # ε = k × IQR

# Sliding windows (used later)
WIN_SIZE = 400
WIN_STEP = WIN_SIZE // 2

# Echo parameters (divergence-based)
PHASE_EPS   = 1e-3   # tiny phase kick for echo
ECHO_FRAC   = 0.5    # half-life at 50% of total divergence
ECHO_MINLEN = 50     # minimum points needed to evaluate half-life



### 3) Utilities: seeding, binarize, LZ, IQR, RR, windows, OLS

In [3]:

def set_seeds(seed: int) -> None:
    import random
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)

def binarize_by_median(x: np.ndarray) -> np.ndarray:
    return (x > np.median(x)).astype(np.uint8)

def lz_entropy_rate(bits: np.ndarray) -> float:
    """Compression-based LZ rate in bits per symbol (for 0/1 arrays)."""
    b = np.asarray(bits, dtype=np.uint8).ravel()
    pad = (-len(b)) % 8
    if pad:
        b = np.concatenate([b, np.zeros(pad, dtype=np.uint8)])
    packed = np.packbits(b)
    comp = zlib.compress(packed.tobytes(), level=9)
    return (8.0 * len(comp)) / (len(b) - pad + 1e-12)

def iqr(a: np.ndarray) -> float:
    q1, q3 = np.percentile(a, [25, 75])
    return float(q3 - q1)

def recurrence_rates_iqr(x: np.ndarray,
                         scales: Iterable[float] = RR_SCALES,
                         max_points: int = 3000,
                         seed: int = 0) -> Dict[str, float]:
    """RR(ε) where ε = k × IQR; uses subsampling for large arrays."""
    rng = np.random.default_rng(seed)
    x = np.asarray(x, dtype=float)
    xs = x[rng.choice(len(x), size=max_points, replace=False)] if len(x) > max_points else x
    D = np.abs(xs[:, None] - xs[None, :])
    diffs = D[~np.eye(len(xs), dtype=bool)]
    scale = max(iqr(xs), 1e-12)
    out = {}
    for s in scales:
        eps = s * scale
        out[f"RR_iqr{s:.1f}"] = float((diffs < eps).mean())
    return out

def sliding_windows(x: np.ndarray, win: int = WIN_SIZE, step: int = WIN_STEP) -> Iterator[Tuple[int, np.ndarray]]:
    """Yield (start_index, window_array)."""
    n = len(x)
    if n < win:
        yield 0, x.copy()
    else:
        for start in range(0, n - win + 1, step):
            yield start, x[start:start + win]

def ols_slope(x: Iterable[float], y: Iterable[float]) -> float:
    """Return OLS slope of y ~ x (no intercept)."""
    x = np.asarray(list(x), dtype=float)
    y = np.asarray(list(y), dtype=float)
    xm = x - x.mean()
    ym = y - y.mean()
    return float((xm * ym).sum() / ((xm * xm).sum() + 1e-12))



### 4) Hook in your physical simulator & echo (divergence‑based)

Replace the placeholder with your real simulator. You only need to return a 1D array x of length T_STEPS for given (flux, seed).

In [4]:
# 4) Simulator & echo (divergence-based)
def simulate_signal(flux: float, seed: int, T: int, *,
                    phase_offset: float = 0.0, noise_std: float = 0.03) -> np.ndarray:
    """
    Surrogate 'physical' signal:
    - Effective frequency increases with flux.
    - Optional phase_offset (for echo).
    - Optional additive Gaussian noise (kept ON for black-box metrics; OFF for echo).
    """
    set_seeds(seed)
    t = np.arange(T, dtype=float)
    f0, gain = 0.01, 6.0
    freq = f0 + gain * flux
    sig = np.sin(freq * t + phase_offset) + 0.3 * np.sin(2 * freq * t + 0.4 + 2 * phase_offset)
    sig = np.tanh(1.2 * sig)
    if noise_std:
        sig = sig + noise_std * np.random.randn(T)
    return sig.astype(float)



### 5) Phase & spectral sensitivity helpers (robust for oscillatory signals)

In [5]:

import numpy as np
from scipy.signal import butter, filtfilt, hilbert

def _peak_omega(x, fs=1.0):
    X = np.abs(np.fft.rfft(x - x.mean()))**2
    f = np.fft.rfftfreq(len(x), d=1/fs)
    k = int(np.argmax(X[1:])) + 1   # ignore DC
    return 2*np.pi*f[k]

def _lp(y, cutoff, fs=1.0, order=4):
    b, a = butter(order, cutoff/(0.5*fs), btype="low")
    return filtfilt(b, a, y)

def _baseband_phase(x, omega0, fs=1.0):
    t = np.arange(len(x))/fs
    z = (x - x.mean()) * np.exp(-1j*omega0*t)  # heterodyne to baseband
    zr = _lp(np.real(z), cutoff=0.15*omega0/(2*np.pi), fs=fs)
    zi = _lp(np.imag(z), cutoff=0.15*omega0/(2*np.pi), fs=fs)
    return np.unwrap(np.angle(hilbert(zr)))  # analytic phase of basebanded real part

def phi_dephase_time_demod(flux, seed, T, rel_eps=0.01, fs=1.0, thresh=np.pi/2):
    """
    Phase dephasing time between base freq ω0 and slightly higher (1+rel_eps)*ω0.
    Returns index of first |Δφ| >= thresh; NaN if it never crosses.
    """
    # base signal (noise-free for clean phase)
    x0 = simulate_signal(flux, seed, T, noise_std=0.0)
    omega0 = _peak_omega(x0, fs=fs)

    # compute flux' that yields (1+rel_eps)*omega0  [invert freq->flux]
    f0, gain = 0.01, 6.0
    flux2 = ((omega0*(1.0+rel_eps))/(2*np.pi) - f0) / gain
    x1 = simulate_signal(flux2, seed, T, noise_std=0.0)

    phi0 = _baseband_phase(x0, omega0, fs=fs)
    phi1 = _baseband_phase(x1, omega0*(1.0+rel_eps), fs=fs)
    dphi = np.abs(phi1 - phi0)
    i = int(np.argmax(dphi >= thresh))
    return float(i) if i > 0 and dphi[i] >= thresh else float('nan')

def spectral_fwhm_fft(x: np.ndarray, fs: float = 1.0) -> float:
    x = np.asarray(x, dtype=float); x = x - x.mean()
    P = np.abs(np.fft.rfft(x))**2
    f = np.fft.rfftfreq(len(x), d=1.0/fs)
    i0 = int(np.argmax(P))
    if i0 == 0 or i0 == len(P)-1:
        return float('nan')
    half = P[i0] / 2.0
    il = i0
    while il > 0 and P[il] > half: il -= 1
    ir = i0
    while ir < len(P)-1 and P[ir] > half: ir += 1
    return float(max(f[ir] - f[il], 0.0))

def ifreq_spread_from_phase(x: np.ndarray) -> float:
    phi = np.unwrap(np.angle(hilbert(x - x.mean())))
    return float(np.std(np.diff(phi)))


In [6]:
# --- Sensitivity metrics that work on oscillatory signals ---
from scipy.signal import hilbert  # uses SciPy; already in your environment

def _analytic_phase(x: np.ndarray) -> np.ndarray:
    return np.unwrap(np.angle(hilbert(x)))

def phase_dephasing_time_relative(flux: float, seed: int, T: int,
                                  rel_eps: float = 0.01, thresh: float = np.pi/2) -> float:
    """
    Reference: noise-free signal at base freq.
    Perturbed: same signal with a slight relative frequency increase (1+rel_eps).
    Returns first index where |Δφ(t)| >= thresh; NaN if it never crosses.
    """
    # simulate base
    x0 = simulate_signal(flux, seed, T, noise_std=0.0)
    # compute flux' that yields base_freq * (1+rel_eps)
    f0, gain = 0.01, 6.0
    base_freq = f0 + gain*flux
    flux2 = (base_freq*(1.0 + rel_eps) - f0) / gain
    x1 = simulate_signal(flux2, seed, T, noise_std=0.0)

    phi0 = _analytic_phase(x0)
    phi1 = _analytic_phase(x1)
    dphi = np.abs(phi1 - phi0)
    i = int(np.argmax(dphi >= thresh))
    return float(i) if dphi[i] >= thresh else float('nan')

def spectral_fwhm_fft(x: np.ndarray, fs: float = 1.0) -> float:
    """
    FFT-based dominant-peak FWHM (no SciPy needed beyond numpy).
    Returns width in frequency units.
    """
    x = np.asarray(x, dtype=float)
    x = x - x.mean()
    P = np.abs(np.fft.rfft(x))**2
    f = np.fft.rfftfreq(len(x), d=1.0/fs)

    i0 = int(np.argmax(P))
    if i0 == 0 or i0 == len(P)-1:
        return float('nan')
    half = P[i0] / 2.0

    # walk left to half
    il = i0
    while il > 0 and P[il] > half:
        il -= 1
    # walk right to half
    ir = i0
    while ir < len(P)-1 and P[ir] > half:
        ir += 1
    return float(max(f[ir] - f[il], 0.0))

def ifreq_spread_from_phase(x: np.ndarray) -> float:
    """Std of instantaneous frequency (first difference of analytic phase)."""
    phi = _analytic_phase(x)
    return float(np.std(np.diff(phi)))


### 6) Run one condition + batch sweep

In [7]:
# 6) Run one condition + batch sweep (updated)
def run_condition(flux: float, seed: int, T: int = T_STEPS) -> dict:
    # --- Sensitivity metrics on clean / analytic signals ---
    phi_t = phi_dephase_time_demod(flux, seed, T, rel_eps=0.01, fs=1.0, thresh=np.pi/2)

    # --- Black-box metrics on realistic (noisy) trajectory ---
    x = simulate_signal(flux, seed, T, noise_std=0.03)
    bits = binarize_by_median(x)
    lz   = lz_entropy_rate(bits)
    rr   = recurrence_rates_iqr(x, scales=RR_SCALES, seed=seed)

    # --- Spectral/IF sensitivity on the same noisy trajectory ---
    fwhm = spectral_fwhm_fft(x, fs=1.0)
    ifsd = ifreq_spread_from_phase(x)

    return {"flux": flux, "seed": seed,
            "lz_rate": lz, **rr,
            "phi_t_half": phi_t,       # expect ↓ with flux
            "psd_fwhm": fwhm,          # expect ↑ with flux
            "ifreq_spread": ifsd}      # expect ↑ with flux


# Batch sweep
rows = []
for f in FLUX_LEVELS:
    for s in SEEDS:
        m = run_condition(f, s, T_STEPS)
        rows.append({"timestamp": time.strftime("%Y-%m-%d %H:%M:%S"),
                     "T_STEPS": T_STEPS, **m})
df = pd.DataFrame(rows)
raw_csv = OUTPUTS / "v021_physical_results.csv"
df.to_csv(raw_csv, index=False)
print("Saved:", raw_csv)
df.head()



Saved: /Users/nadjamuller/number1/setup/outputs/v021_physical_results.csv


,timestamp,T_STEPS,flux,seed,lz_rate,RR_iqr0.1,RR_iqr0.3,RR_iqr0.5,phi_t_half,psd_fwhm,ifreq_spread
0,2025-11-10 22:23:01,9000,0.0000,7,0.118222,0.169094,0.360745,0.513248,386.0,0.000222,0.049478
1,2025-11-10 22:23:02,9000,0.0000,21,0.106667,0.171503,0.367259,0.516727,386.0,0.000222,0.050727
2,2025-11-10 22:23:02,9000,0.0000,37,0.107556,0.166447,0.362048,0.514303,386.0,0.000222,0.050813
3,2025-11-10 22:23:02,9000,0.0002,7,0.113778,0.160130,0.356381,0.508807,774.0,0.000222,0.048421
4,2025-11-10 22:23:02,9000,0.0002,21,0.112889,0.162240,0.358586,0.509081,774.0,0.000222,0.049754


### 7) Flux‑level summary + line/overlay plots

In [8]:
summary = (df.groupby("flux")[["lz_rate","RR_iqr0.1","RR_iqr0.3","RR_iqr0.5",
                               "phi_t_half","psd_fwhm","ifreq_spread"]]
           .mean().reset_index())
summary.to_csv(OUTPUTS / "v021_summary.csv", index=False)

def save_line(x, y, title, ylab, fname):
    fig, ax = plt.subplots()
    ax.plot(x, y, marker="o")
    ax.set_title(title); ax.set_xlabel("Flux"); ax.set_ylabel(ylab)
    fig.tight_layout(); fig.savefig(OUTPUTS / fname, dpi=300); plt.close(fig)

save_line(summary["flux"], summary["phi_t_half"],   "Phase dephasing time vs flux", "phi_t_half",   "v021_plot_phi_half.png")
save_line(summary["flux"], summary["psd_fwhm"],     "Spectral FWHM vs flux",        "psd_fwhm",     "v021_plot_fwhm.png")
save_line(summary["flux"], summary["ifreq_spread"], "IF spread vs flux",            "ifreq_spread", "v021_plot_ifspread.png")

for col in ["phi_t_half","psd_fwhm","ifreq_spread","lz_rate","RR_iqr0.1","RR_iqr0.3","RR_iqr0.5"]:
    # overlays
    fig, ax = plt.subplots()
    for s in sorted(df["seed"].unique()):
        d = df[df["seed"]==s].sort_values("flux")
        ax.plot(d["flux"], d[col], marker="o", label=f"seed {s}")
    ax.set_title(col); ax.set_xlabel("Flux"); ax.set_ylabel(col); ax.legend()
    fig.tight_layout(); fig.savefig(OUTPUTS / f"v021_overlay_{col}.png", dpi=300); plt.close(fig)



### 8) Sliding‑window trends (sanity check)

In [9]:
# 8) Sliding-window metrics (LZ + RR in windows)

win_rows = []
for f in FLUX_LEVELS:
    for s in SEEDS:
        x = simulate_signal(f, s, T_STEPS, noise_std=0.03)
        for start, xw in sliding_windows(x, WIN_SIZE, WIN_STEP):
            bits = binarize_by_median(xw)
            lz  = lz_entropy_rate(bits)
            rr  = recurrence_rates_iqr(xw, scales=RR_SCALES, seed=s)
            win_rows.append({"flux": f, "seed": s, "start": start,
                             "lz": lz, "RR_iqr0.1": rr["RR_iqr0.1"],
                             "RR_iqr0.3": rr["RR_iqr0.3"], "RR_iqr0.5": rr["RR_iqr0.5"]})

win_df = pd.DataFrame(win_rows)
win_csv = OUTPUTS / "window_trends.csv"
win_df.to_csv(win_csv, index=False)
print("Saved:", win_csv)


Saved: /Users/nadjamuller/number1/setup/outputs/window_trends.csv


### 9) Light RQA: DET & Lmax from a recurrence plot

In [10]:


def rqa_det_lmax(x: np.ndarray, eps_scale: float = 0.3, tau: int | None = None, m: int = 3) -> tuple[float, float]:
    """
    Minimal RQA:
      - delay tau: first ACF drop below 1/e (fallback=10)
      - embedding m
      - threshold ε = eps_scale * IQR(x)
      - returns (DET, Lmax)
    """
    x = np.asarray(x, dtype=float)
    if tau is None:
        # crude ACF-based tau
        maxlag = min(500, len(x)//4)
        mu, var = x.mean(), x.var() + 1e-12
        acf = [np.correlate(x[:len(x)-k]-mu, x[k:]-mu)[0] / ((len(x)-k)*var) for k in range(1, maxlag)]
        below = np.where(np.array(acf) < 1/np.e)[0]
        tau = int(below[0] + 1) if len(below) else 10

    N = len(x) - (m-1)*tau
    if N <= 50:
        return np.nan, np.nan

    X = np.column_stack([x[i:i+N] for i in range(0, m*tau, tau)])
    eps = max(iqr(x), 1e-12) * eps_scale
    D = np.sqrt(((X[:, None, :] - X[None, :, :])**2).sum(axis=2))
    R = (D < eps).astype(np.uint8)

    # diagonal line lengths (i+1, j+1) continuation
    n = R.shape[0]
    Ls = []
    for i in range(n-1):
        run = 0
        for j in range(n-1):
            if R[i, j] and R[i+1, j+1]:
                run += 1
            else:
                if run >= 2: Ls.append(run)
                run = 0
        if run >= 2: Ls.append(run)

    if not Ls:
        return 0.0, 0.0
    Ls = np.array(Ls, dtype=float)
    DET  = float(Ls.sum() / (R.sum() + 1e-12))
    Lmax = float(Ls.max())
    return DET, Lmax

rqa_rows = []
for f in FLUX_LEVELS:
    for s in SEEDS:
        x = simulate_signal(f, s, T_STEPS, noise_std=0.03)
        det, lmax = rqa_det_lmax(x, eps_scale=0.3, m=3)
        rqa_rows.append({"flux": f, "seed": s, "DET": det, "Lmax": lmax})

rqa_df = pd.DataFrame(rqa_rows)
rqa_csv = OUTPUTS / "rqa_summary.csv"
rqa_df.to_csv(rqa_csv, index=False)
print("Saved:", rqa_csv)


Saved: /Users/nadjamuller/number1/setup/outputs/rqa_summary.csv


### 10) Robustness stress‑tests (noise/amplitude/downsampling/binarization)

In [11]:


def stress_variants(x: np.ndarray) -> dict[str, np.ndarray]:
    return {
        "base": x,
        "+3dB": x * (10**(3/20)),
        "0.8x": 0.8 * x,
        "down2": x[::2],
        "bin_mean": (x > x.mean()).astype(float),
    }

rob_rows = []
for f in FLUX_LEVELS:
    for s in SEEDS:
        x = simulate_signal(f, s, T_STEPS, noise_std=0.03)
        for name, sig in stress_variants(x).items():
            if sig.dtype != float:  # 'bin_mean' already 0/1
                bits = sig.astype(np.uint8)
                lz = lz_entropy_rate(bits)
                rr = recurrence_rates_iqr(sig, scales=RR_SCALES, seed=s)
            else:
                bits = binarize_by_median(sig)
                lz = lz_entropy_rate(bits)
                rr = recurrence_rates_iqr(sig, scales=RR_SCALES, seed=s)
            rob_rows.append({"flux": f, "seed": s, "variant": name,
                             "lz_rate": lz, **rr})

rob_df = pd.DataFrame(rob_rows)
rob_csv = OUTPUTS / "robustness_summary.csv"
rob_df.to_csv(rob_csv, index=False)
print("Saved:", rob_csv)


Saved: /Users/nadjamuller/number1/setup/outputs/robustness_summary.csv


### 11) Stats & decision gates

In [12]:
def slope_p_with_seed_effect(df_in: pd.DataFrame, ycol: str) -> tuple[float, float]:
    try:
        import statsmodels.formula.api as smf
        d = df_in.copy(); d["seed"] = d["seed"].astype("category")
        model = smf.ols(f"{ycol} ~ flux + seed", data=d).fit()
        return float(model.params.get("flux", np.nan)), float(model.pvalues.get("flux", np.nan))
    except Exception:
        # fallback: average per-seed slopes (no p-value)
        slopes = []
        for s in sorted(df_in["seed"].unique()):
            dd = df_in[df_in["seed"]==s].sort_values("flux")
            slopes.append(ols_slope(dd["flux"], dd[ycol]))
        return float(np.mean(slopes)), float("nan")

tests = {
    "lz_rate":      +1,   # expect up
    "RR_iqr0.1":    -1,   # expect down
    "RR_iqr0.3":    -1,
    "RR_iqr0.5":    -1,
    "phi_t_half":   -1,   # expect down
    "psd_fwhm":     +1,   # expect up
    "ifreq_spread": +1,   # expect up
}

res = {}
for metric, sign in tests.items():
    mdf = df[["flux","seed",metric]].dropna()
    if len(mdf)==0:
        res[metric] = {"slope": np.nan, "p": np.nan, "ok": False}
        continue
    slope, p = slope_p_with_seed_effect(mdf, metric)
    ok = (np.sign(slope) == np.sign(sign)) and (abs(slope) > 0) and (np.isnan(p) or p < 0.05)
    res[metric] = {"slope": slope, "p": p, "ok": ok}

lz_ok  = res["lz_rate"]["ok"]
rr_ok  = sum(res[k]["ok"] for k in ["RR_iqr0.1","RR_iqr0.3","RR_iqr0.5"]) >= 2
sens_ok = (res["phi_t_half"]["ok"] and (res["psd_fwhm"]["ok"] or res["ifreq_spread"]["ok"]))

decision = "SUPPORTED" if (lz_ok and rr_ok and sens_ok) else "INCONCLUSIVE"

lines = [
    f"- LZ slope: {res['lz_rate']['slope']:.4g}, p={res['lz_rate']['p']:.3g}, pass={lz_ok}",
    f"- RR pass≥2/3: {rr_ok} (rr01={res['RR_iqr0.1']['ok']}, rr03={res['RR_iqr0.3']['ok']}, rr05={res['RR_iqr0.5']['ok']})",
    f"- φ-half slope: {res['phi_t_half']['slope']:.4g}, p={res['phi_t_half']['p']:.3g}, pass={res['phi_t_half']['ok']}",
    f"- FWHM slope: {res['psd_fwhm']['slope']:.4g}, p={res['psd_fwhm']['p']:.3g}, pass={res['psd_fwhm']['ok']}",
    f"- IF-spread slope: {res['ifreq_spread']['slope']:.4g}, p={res['ifreq_spread']['p']:.3g}, pass={res['ifreq_spread']['ok']}",
    f"\n**Decision (v0.2.1): {decision}**"
]

with open(OUTPUTS / "v021_stats_summary.md", "w") as f:
    f.write("# Stats summary (v0.2.1)\n\n")
    f.write("\n".join(lines) + "\n")

print("\n".join(lines))


- LZ slope: 3.305, p=nan, pass=True
- RR pass≥2/3: True (rr01=True, rr03=True, rr05=True)
- φ-half slope: -4.827e+04, p=nan, pass=True
- FWHM slope: 0, p=nan, pass=False
- IF-spread slope: 2.357, p=nan, pass=True

**Decision (v0.2.1): SUPPORTED**


### 12) Small run README (artifact index)

In [13]:


readme = OUTPUTS / "v020_README.md"
with open(readme, "w") as f:
    f.write(f"""# v0.2.0 — Physical surrogate + divergence echo

Artifacts produced:
- v020_physical_results.csv     # raw per (flux, seed): lz, RR_iqr*
- v020_summary.csv              # flux means
- window_trends.csv             # sliding-window LZ & RR
- rqa_summary.csv               # DET, Lmax per (flux, seed)
- robustness_summary.csv        # metrics under stressors
- v020_plot_lz.png / v020_plot_rr*.png / v020_plot_echo_half.png
- v020_overlay_*.png            # overlays by seed
- v020_echo_divergence_seed*.png, v020_echo_divergence_flux*.png
- stats_summary.md              # slopes, p-values, pass/fail
""")
print("Wrote:", readme)


Wrote: /Users/nadjamuller/number1/setup/outputs/v020_README.md
